In [1]:
import json
import os
import sys
import boto3
from utils import bedrock, print_ww
from utils.utils import extract_model_info

## Setting up Bedrock API.

In [6]:
boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
    runtime=False
)

Create new client
  Using region: None
boto3 Bedrock client successfully created!
bedrock(https://bedrock.us-east-1.amazonaws.com)


### Let's take a look at which models we have access to.

In [7]:
all_models = boto3_bedrock.list_foundation_models()
print(f"extract_model_info(all_models): {extract_model_info(all_models)}")

extract_model_info(all_models): [('Titan Text Large', 'amazon.titan-tg1-large'), ('Titan Text Embeddings', 'amazon.titan-e1t-medium'), ('Titan Text Embeddings v2', 'amazon.titan-embed-g1-text-02'), ('Titan Text G1 - Express', 'amazon.titan-text-express-v1'), ('Titan Embeddings G1 - Text', 'amazon.titan-embed-text-v1'), ('Stable Diffusion XL', 'stability.stable-diffusion-xl'), ('Stable Diffusion XL', 'stability.stable-diffusion-xl-v0'), ('J2 Grande Instruct', 'ai21.j2-grande-instruct'), ('J2 Jumbo Instruct', 'ai21.j2-jumbo-instruct'), ('Jurassic-2 Mid', 'ai21.j2-mid'), ('Jurassic-2 Mid', 'ai21.j2-mid-v1'), ('Jurassic-2 Ultra', 'ai21.j2-ultra'), ('Jurassic-2 Ultra', 'ai21.j2-ultra-v1'), ('Claude Instant', 'anthropic.claude-instant-v1'), ('Claude', 'anthropic.claude-v1'), ('Claude V1 100k', 'anthropic.claude-v1-100k'), ('Claude', 'anthropic.claude-v2'), ('Claude V2 100k', 'anthropic.claude-v2-100k'), ('Command', 'cohere.command-text-v14')]


In [8]:
bedrock_runtime = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

Create new client
  Using region: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)


In [9]:
prompt_data = """
Human: Tell me everything you know about JSON schema.

Assistant:"""

In [10]:
body = json.dumps({"prompt": prompt_data, "max_tokens_to_sample": 500})
modelId = "anthropic.claude-v2" 
accept = "application/json"
contentType = "application/json"

In [11]:
response = bedrock_runtime.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())

print(response_body.get("completion"))

 Here is a summary of what I know about JSON schema:

- JSON schema is a vocabulary that allows you to annotate and validate JSON documents. It defines a JSON-based format to describe the structure of JSON data.

- JSON schemas are JSON documents themselves that describe JSON data using validation keywords like "type", "properties", "required", etc. These keywords provide rules that JSON documents being validated must adhere to.

- A JSON schema can be used to validate if a JSON document follows the schema, which is useful for data validation and clarifying expectations for JSON data you are consuming or producing. 

- Some common things JSON schema validation checks:
    - Valid data types (string, number, boolean, etc)
    - Required properties
    - String length limits
    - Value ranges for numbers
    - Expected object structure/hierarchy
    - Referencing definitions for reused JSON structures

- There are several JSON schema specification versions (draft-04, draft-06, draft-07,

In [28]:
prompt_data = """
Human: Your role is to form function arguments based on the specification provide in 'FUNCTIONS'.  Users will ask you a question about the weather, and you'll attempt to form the function call that we can pass along to a tool to determine the response. Do not try to answer questions about the weather directly, use the provided tools.  The tools are specified using a json schema format, which defines how the arguments to the function are to be formed.

FUNCTIONS = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "format": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"],
                    "description": "The temperature unit to use. Infer this from the users location.",
                },
            },
            "required": ["location", "format"],
        },
    }
]

Examples
###
Human: 
What's the weather like in Glasgow Scotland?
Assistant:
{
'function_call': {'name': 'get_current_weather',
    'arguments': {
        "location": "Glasgow, Scotland",
        "format": "celsius"
    }
}

Human: 
What's the weather like in Raleigh, NC?
Assistant:
{
'function_call': {'name': 'get_current_weather',
    'arguments': {
        "location": "Raleigh, North Carolina",
        "format": "fahrenheit"
    }
}
###

Human:
What's the weather like in Washington, DC?

Assistant:
"""

In [29]:
body = json.dumps({"prompt": prompt_data, "max_tokens_to_sample": 500})
modelId = "anthropic.claude-v2" 
accept = "application/json"
contentType = "application/json"

In [30]:
response = bedrock_runtime.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())

print(response_body.get("completion"))

 Here is the function call to get the current weather in Washington, DC:

{
  "function_call": {
    "name": "get_current_weather", 
    "arguments": {
      "location": "Washington, DC",
      "format": "fahrenheit"
    }
  }
}


# And Now for the Main Event.

Here's a class that prompts a language model for function arguments for a linear regression, based on data that we load into the context from and external source.


In [18]:
from prompt_template import PromptTemplate, load_text_file
from function_parser import FunctionParser
from linear_trend_model import run_linear_trend_model, FUNCTIONS

In [20]:
#: Here's our prompt that we'll use to drive the language model
linear_trend_prompt = load_text_file('linear_trend_request.prompt')

#: Here's a mock article excerpt that I generated to showcase this technology.
city_dogs =  load_text_file('city_dogs.txt')


In [22]:
#: Establish the Prompt template from the text file.
linear_trend_prompt_template = PromptTemplate(linear_trend_prompt)

#: Fill the prompt using our 
linear_trend_filled_prompt = linear_trend_prompt_template.fill(
    FUNCTIONS=str(FUNCTIONS), 
    user_data=city_dogs
)
print(f"linear_trend_filled_prompt: {linear_trend_filled_prompt}")

linear_trend_filled_prompt: Human:

You will play the role of an expert Data Scientist assistant.  You love linear models, and when you get linear data you make a function call to fit a model to the data. We'll use triple hash ### to open and close subsections of prompt examples.

We're going to define your character and behaviour in terms of the red, yellow, blue, and green color categories that relate to personality types.

The color system I'm referring to - it's a way of categorizing personality traits into four colors:

Red personalities are usually seen as bold, expressive, and competitive. They are motivated by power, stimulation, and winning.
Yellow personalities are sociable, outgoing, and enthusiastic. They tend to be persuasive, verbal, and eager to collaborate.
Green personalities are calm, accommodating, and caring. They value stability, compassion, and building community.
Blue personalities are logical, detail-oriented, and focused. They are quantitative, deliberate, and 

In [23]:
body = json.dumps({"prompt": linear_trend_filled_prompt, "max_tokens_to_sample": 500})
modelId = "anthropic.claude-v2" 
accept = "application/json"
contentType = "application/json"

In [24]:
response = bedrock_runtime.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())

print(response_body.get("completion"))

 Here is the function call to run a linear regression on the data provided:

```json
{
  "function": "run_linear_trend_model", 
  "parameters": {
    "x": [0.5, 1.5, 3, 4.2, 6],
    "y": [15000, 12500, 10000, 7500, 6000]
  }
}
```

I extracted the distances in miles from Times Square for each location as the x values, and the monthly revenues in dollars for each location as the y values. This allows fitting a linear regression model to analyze the relationship between distance from Times Square and revenue.


In [ ]:
parser = FunctionParser()